# Problem 9: Special Pythagorean Triplet
![problem](img/p0009.png)

## Conventions
- a, b, c are the sides of the triangle
- S is the perimeter of the triangle

## Naive Approach
Without involvement of any theorems, or any specialised formula, we can start a brute-force search to see which combination will satisfy Pythagoras Triplet:
- Triangles have 3 sides (a, b, c)
- c > a > b

Without further ado, we can generate every possible values of a, b, and c, and check if the sum of squares of a and b is equal to the square of c.

In [1]:
def q9_brute_force(perimeter):
	for b in range(1, perimeter + 1):
		for a in range(b, perimeter + 1):
			for c in range(a, perimeter + 1):
				if (new_S := (a + b + c)) > 1000: break
				if new_S != 1000: continue

				if a ** 2 + b ** 2 == c ** 2:
					print(f"a={a}, b={b}, c={c}")
					return a * b * c


from solutions.euler.util.decorators import timed_function

assert (timed_function(q9_brute_force)(1000) == 31875000)


a=375, b=200, c=425
q9_brute_force(1000)=31875000
Took 918.0018901824951 ms


## optimisations
We should try and narrow down the search space. The obvious one is instead of trying different values of c (or whichever the inner-most value) we can derive it. In this example, we use c, but we can use a = 1000 - b - c and b = 1000 - a - c
$
a + b + c = 1000 \\
\hspace{2.4em} \quad c = 1000 - a - b
$

Also, we can also restrict a given that we know that:
$
a + b + c = S \\
c > b >= a \\
a + b > c
$ 
(a + b) has to be bigger than c (it is a triangle after all). Therefore, maximum value of c is S / 2, but it also cannot be smaller than S / 3. If so, (let's say S / 3 - 1), a and b has to be S / 3 - 2 or smaller, which added up will not make up S.   

Let's restrict a (the smaller of the remaining a, b). a + b = S - c. And if we assume that b is the larger of the 2, the largest a can be is (S - c) / 2. But it cannot be so small such that it makes b bigger than c. Constraining range of b allows us to constraint range of a, that helps us narrow down the search-space.
$
b < c \\
b = S - c - a \\
S - c - a < c \\
S - 2c < a \\
$

In [2]:
def q9_improved(perimeter=1000):
	min_c, max_c = perimeter // 3, perimeter // 2
	logging.debug("min_c=%s, max_c=%s", min_c, max_c)

	for c in range(min_c, max_c + 1):
		min_a, max_a = perimeter - (2 * c) + 1, min(perimeter // 3, (perimeter - c) // 2)
		logging.debug("min_a=%s, max_a=%s", min_a, max_a)

		for a in range(min_a, max_a + 1):
			b = perimeter - a - c

			if a * a + b * b == c * c:
				logging.info("S=%s::%s", perimeter, (a, b, c))
				return a * b * c

	raise ValueError("No solution found")


import logging
import sys

from solutions.euler.util.decorators import timed_function

logging.basicConfig(stream=sys.stderr, level=logging.INFO)
assert timed_function(q9_improved)() == 31875000

INFO:root:S=1000::(200, 375, 425)


q9_improved()=31875000
Took 0.9052753448486328 ms


# Euclid's Formula
Then comes euclid's formula. We can rewrite a, b, c such that

$
a = m^2 - n^2\\
b = 2mn\\
c = m^2 + n^2
$

which will satisfy the original pythagoras equation $a^2 + b^2 = c^2$. Here comes the genius part -- if $m, n$ are all coprime, the triple $a, b, c$ generated will all be coprime also. And it can be shown that coprime pairs of m, n that only 1 of them are odd will generate all primitive pythagorean triples. They can then be multiplied out (by k) to generate non-primitive pythagorean triples.

In [3]:
from solutions.revisit.p9 import generate_coprime_pairs


def euclid_formula(upper_bound):
	for m, n in generate_coprime_pairs(upper_bound=int(upper_bound ** 0.5) + 1):
		if 1 != (number_of_odd := len(list(filter(lambda x: x % 2 == 0, (m, n))))):
			continue

		a = m * m - n * n
		b = 2 * m * n
		c = m * m + n * n

		assert a * a + b * b == c * c
		for k in range(1, (upper_bound // (a + b + c)) + 1):
			if k == 1:
				print(f"Primitive Triplet: {a, b, c}, m,n= {m, n}")

			ka, kb, kc = k * a, k * b, k * c
			print(f"  - {ka, kb, kc}")


N = 100
print(f"Generating all pythagorean triple that has perimeter < {N}")
euclid_formula(N)

Generating all pythagorean triple that has perimeter < 100
Primitive Triplet: (3, 4, 5), m,n= (2, 1)
  - (3, 4, 5)
  - (6, 8, 10)
  - (9, 12, 15)
  - (12, 16, 20)
  - (15, 20, 25)
  - (18, 24, 30)
  - (21, 28, 35)
  - (24, 32, 40)
Primitive Triplet: (5, 12, 13), m,n= (3, 2)
  - (5, 12, 13)
  - (10, 24, 26)
  - (15, 36, 39)
Primitive Triplet: (15, 8, 17), m,n= (4, 1)
  - (15, 8, 17)
  - (30, 16, 34)
Primitive Triplet: (7, 24, 25), m,n= (4, 3)
  - (7, 24, 25)
Primitive Triplet: (21, 20, 29), m,n= (5, 2)
  - (21, 20, 29)
Primitive Triplet: (9, 40, 41), m,n= (5, 4)
  - (9, 40, 41)
Primitive Triplet: (35, 12, 37), m,n= (6, 1)
  - (35, 12, 37)


## Improving Euclid's formula?
Question -- by generating corpime pairs of $m,n$ (and also satisfy only 1 of them are odd) we can generate all primitive pythagoras triplet. This in turn by multiplying k, we can generate all pythagorean triples.

Can we skip the co-prime pair generation (of $m, n$) by iterating through all numbers of $m, n$?

In [4]:
def euclid_formula_2(upper_bound):
	all_triples = set()

	for m in range(2, upper_bound):
		for n in range(1, m):
			a = m * m - n * n
			b = 2 * m * n
			c = m * m + n * n

			if (perimeter := a + b + c) > upper_bound:
				break

			all_triples.add(tuple(sorted((a, b, c))))

	return all_triples


def euclid_formula(upper_bound):
	all_triples = set()

	for m, n in generate_coprime_pairs(upper_bound=int(upper_bound ** 0.5) + 1):
		if 1 != (number_of_odd := len(list(filter(lambda x: x % 2 == 0, (m, n))))):
			continue

		a = m * m - n * n
		b = 2 * m * n
		c = m * m + n * n

		for k in range(1, (upper_bound // (a + b + c)) + 1):
			ka, kb, kc = k * a, k * b, k * c
			if (ka + kb + kc) <= upper_bound:
				all_triples.add(tuple(sorted((ka, kb, kc))))

	return all_triples


print(f'Simpler: {sorted(euclid_formula_2(50))}')
print(f'Primitive: {sorted(euclid_formula(50))}')

Simpler: [(3, 4, 5), (5, 12, 13), (6, 8, 10), (8, 15, 17), (12, 16, 20)]
Primitive: [(3, 4, 5), (5, 12, 13), (6, 8, 10), (8, 15, 17), (9, 12, 15), (12, 16, 20)]


We can see that the difference is (9, 12, 15).
$
a = m^2 - n^2 \\
c = m^2 + n^2 \\
a + c = 2*m^2 \\
9 + 15 = 2*m^2 \\
24 = 2 * m ^ 2 \\
\sqrt{12} = m
$
We can see that m has to be irrational to create this pair. This (by approach) shows that we need coprime pairs to generate all primitive triples (and then to multiply them out)

# HackerRank
So HackerRank has Project Euler also, which allows us test against more inputs. Project Euler checks our solution against only 1 set of input (and output) so we can have got-to-solution situation, but wasn't functionally correct situation.

q9_improved timed out as the complexity is quadratic (although bound), and so when perimeter becomes sufficiently large, it starts failing

euclid_formula version passes, although it has a pre-compute phase, which is fairly expensive. I'm sure the search space could be tightened (by examining the input, and only computing relevant "ranges"), but I didn't feel the need to. It passes, but not the fastest solution.

Then I came across another solution on HackerRank (publicly available anyway) so I enclose it here also. 

In [5]:
from solutions.revisit.p9 import generate_coprime_pairs

N = 3000


def euclid_formula(upper_bound=N):
	multipliers = {}
	for m, n in generate_coprime_pairs(upper_bound=int(upper_bound ** 0.5) + 1):
		if 1 != (number_of_odd := len(list(filter(lambda x: x % 2 == 0, (m, n))))):
			continue

		a = m * m - n * n
		b = 2 * m * n
		c = m * m + n * n

		assert a * a + b * b == c * c
		for k in range(1, (upper_bound // (a + b + c)) + 1):
			ka, kb, kc = k * a, k * b, k * c
			kS, kM = ka + kb + kc, ka * kb * kc

			multipliers[kS] = max(multipliers.get(kS, -1), kM)

	return multipliers


def reference_solution(n):
	"""
	This is the reference solution I found online in HackerRank, that uses clever optimisation
	https://www.hackerrank.com/contests/projecteuler/challenges/euler009/forum/comments/1136234
		Explanation of it is in the cell below
	"""
	if n % 2 != 0:
		return -1

	m = -1
	for a in range(1, (n // 3) + 1):
		b = n * (n - 2 * a) // (2 * (n - a))
		c = n - a - b
		if a ** 2 + b ** 2 == c ** 2:
			m = max(m, a * b * c)

	return m


generated = euclid_formula()
for i in range(N):
	if (result := generated.get(i, -1)) != (answer := reference_solution(i)):
		print(i, result, answer)


### Reference Solution
The credit goes to https://www.hackerrank.com/contests/projecteuler/challenges/euler009/forum/comments/1136234. This is simply a transcription / slightly editing here and there for my own understanding.  

$
a ^ 2 + b ^ 2 = c ^ 2 \\
a^2 = c^2 - b^2 \\
a^2 = (c + b) * (c - b) \\
$

it shows us
$
a^2 = (n - a) * (c - b)
$

if $c$ and $b$ must be integer then $ a^2 % (n - a) $ must be zero (otherwise $a$ is not valid)

and we can calculate \( c - b \) by 
$ \frac{a^2}{n - a} = \text{diff} $

then we can find \( c \) by

$ 
c + b = n - a \\
c - b = \text{diff} \\
c = \frac{(n - a) + \text{diff}}{2} \quad \text{(c must be an integer, otherwise it is not valid)} 
$

so far we know $ n $, $ a $, and $ c $

$ b = n - a - c $

then we can calculate the product.